In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers

In [2]:
dataframe = pd.read_csv('ageC.csv')

In [3]:
#preprocess here
#Drop un-used columns.
dataframe = dataframe.drop(columns=['fnlwgt', 'class', 'ageC', 'capital-gain', 'capital-loss', 
                                   'capital-lossB'])
dataframe.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,target,capital-gainB
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,1,0
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States,1,0
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,0,0
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States,0,1
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,30,United-States,1,0


In [4]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [5]:
feature_columns = []

# numeric cols
for header in ['age', 'education-num', 'capital-gainB', 'hours-per-week']:
  feature_columns.append(feature_column.numeric_column(header))

# indicator_columns
indicator_column_names = ['workclass', 'marital-status', 'occupation', 'relationship',
                          'race', 'sex', 'native-country']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [6]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(dataframe, test_size=0.2)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

print(len(train))

35166


In [7]:
checkpoint_path = "census/cp-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=0, 
    save_weights_only=True,
    save_freq=1000)


model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='RMSprop',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[cp_callback], verbose=1)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1095/1099 [============================>.] - ETA: 0s - loss: 0.3864 - accuracy: 0.8077WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_7:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'hours-per-week': <tf.

Epoch 35/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3155 - accuracy: 0.8502 - val_loss: 0.3228 - val_accuracy: 0.8468
Epoch 36/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3147 - accuracy: 0.8484 - val_loss: 0.3275 - val_accuracy: 0.8466
Epoch 37/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3147 - accuracy: 0.8492 - val_loss: 0.3245 - val_accuracy: 0.8420
Epoch 38/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3149 - accuracy: 0.8500 - val_loss: 0.3287 - val_accuracy: 0.8348
Epoch 39/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3145 - accuracy: 0.8491 - val_loss: 0.3248 - val_accuracy: 0.8460
Epoch 40/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3143 - accuracy: 0.8493 - val_loss: 0.3216 - val_accuracy: 0.8366
Epoch 41/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3146 - accuracy: 0.8488 - val_loss: 0.3232 -

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3131 - accuracy: 0.8501 - val_loss: 0.3245 - val_accuracy: 0.8429
Epoch 92/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3130 - accuracy: 0.8518 - val_loss: 0.3254 - val_accuracy: 0.8389
Epoch 93/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3121 - accuracy: 0.8495 - val_loss: 0.3274 - val_accuracy: 0.8443
Epoch 94/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3131 - accuracy: 0.8510 - val_loss: 0.3238 - val_accuracy: 0.8366
Epoch 95/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3132 - accuracy: 0.8508 - val_loss: 0.3240 - val_accuracy: 0.8458
Epoch 96/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3130 - accuracy: 0.8514 - val_loss: 0.3302 - val_accuracy: 0.8443
Epoch 97/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3128 - accuracy: 0.8513 - val_loss: 0.3241 - val_accuracy